In [67]:
import numpy as np
import os
import pickle

THETA = 0.01
LEARNING_RATE = 0.01
EPOCHS = 3


In [88]:
def main():
    # load CIFAR-10 train, validation data
    xTrain, yTrain, xValid, yValid = load_train_data("CIFAR-10-train")

    # load CIFAR-10 test data
    xTest, yTest = load_test_data("CIFAR-10-test")

    # initialize a random weight matrix for CIFAR-10
    W = np.random.rand(10, 3073)*0.001

    # train the weights, epoch loop
    for epoch in range(EPOCHS):
        loss, accuracy = SVM_loss(xTrain, yTrain, W, theta=THETA)
        print(
            f'In epoch {epoch}, loss is {loss}\nIn epoch {epoch} accuracy is {accuracy}')

    # eval_gradient(loss_fun,W)


In [53]:
# only used in load_train_data() and load_test_data() to read pickle file
def unpickle(file):
    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


In [82]:
def load_train_data(filename):
    """
    Load CIFAR-10 train data
    xTrain combined image arrays
    yTrain is the vector of correct labels for each image
    """

    # get train data 1-6 append them to a list
    temp_xTrain = []
    temp_yTrain = []
    for file in os.listdir(filename):
        data = unpickle(os.path.join(filename, file))
        temp_xTrain.append(data[b'data'])
        temp_yTrain.append(data[b'labels'])

    # make one list from the list of lists train data is now ready
    xTrain = np.concatenate(temp_xTrain)
    yTrain = np.concatenate(temp_yTrain)

    # change datatype to float
    xTrain = xTrain.astype('float64')

    # normalize data ???
    for id, im in enumerate(xTrain):
        normIm = im/255
        xTrain[id] = normIm

    # center data ???
    avIm = xTrain.mean(axis=0, keepdims=True)
    for id, im in enumerate(xTrain):
        normIm = im-avIm
        xTrain[id] = normIm

    # append bias to images arrays
    xTrain = np.c_[xTrain, np.ones(50000).T]

    # assign 1/5 of the images as validation dataset
    xValid = xTrain[:len(xTrain)//5]
    xTrain = xTrain[len(xTrain)//5:]

    yValid = yTrain[:len(yTrain)//5]
    yTrain = yTrain[len(yTrain)//5:]

    # transpose so xTrain is 3073x40000 yTrains is 40000x1 xValid is 3073x10000 yValid is 10000x1
    xTrain, yTrain, xValid, yValid = xTrain.T, yTrain.T, xValid.T, yValid.T

    return (xTrain, yTrain, xValid, yValid)


In [48]:
def load_test_data(filename):
    """
    Load CIFAR-10 test data
    """
    data = unpickle(os.path.join(filename, os.listdir(filename)[0]))
    xTest = (data[b'data'])
    yTest = (data[b'labels'])
    yTest = np.array(yTest)

    # change datatype to float
    xTest = xTest.astype('float64')

    # normalize data ???
    for id, im in enumerate(xTest):
        normIm = im/255
        xTest[id] = normIm

    # center data ???
    avIm = xTest.mean(axis=0, keepdims=True)
    for id, im in enumerate(xTest):
        normIm = im-avIm
        xTest[id] = normIm

    # append bias to images arrays
    xTest = np.c_[xTest, np.ones(10000).T]

    # transpose so that xTest is 3073x10000 yTest is 10000
    xTest, yTest = xTest.T, yTest.T
    return (xTest, yTest)


In [87]:
def SVM_loss(X, y, W, theta):
    """
    Multiclass Support Vector Machine Loss

    x is matrix of flattened image vectors with appended biases
    y is the integer valued matrix representing correct labels for images
    W is the weight matrix
    theta is the hyperparameter and usually determined by cross-validation

    We set up SVM loss so that for each image,
    its score of correct class is higher than every other
    incorrect class by a fixed margin.

    Multiclass SVM loss is formalized as:

    L = data_loss + theta*regularization_penalty

    wherein 
    data_loss =  (1/numOfTrainingData) *   sum    (max(0, (s_j - s_yi + margin)))
                                         (j != yi)
    (we assign margin value 1 and reduce number of hyperparameters
    because theta and margin are inverse proportional)

    regularization penalty = sum (sum ((W_k,l)**2))
                             (k)   (l)    
    """
    margin = 1

    # weight times images
    score_matrix = W@X

    # get the correct class score for every image
    yi_scores = score_matrix[y, np.arange(score_matrix.shape[1])]
    yi_scores = np.matrix(yi_scores)

    # margins equals to below part, to ignore when s_j == s_yi we overwrite them by assigning 0s
    #    sum    (max(0, (s_j - s_yi + margin)))
    # (j != yi)
    margins = np.maximum(0, score_matrix - yi_scores + margin)
    margins[y, np.arange(X.shape[1])] = 0

    # data loss and regularization part
    data_loss = np.mean(np.sum(margins, axis=0))
    regularization_penalty = np.sum(np.square(W))

    # total loss
    SVM_loss_score = data_loss + theta*regularization_penalty

    # find class where the score is the highest
    yPredict = np.argmax(score_matrix, axis=0)

    # calculate accuracy
    accuracy = np.mean(yPredict == y)
    
    return (SVM_loss_score, accuracy)


In [10]:
def loss_fun(W):
    return SVM_loss(xTrain, yTrain, W, theta=THETA)


In [90]:
def eval_gradient(f, score_matrix, x):
    """
    Analytic gradient of SVM

    L = data_loss + theta*regularization_penalty

    data_loss =  (1/numOfTrainingData) *   sum    (max(0, (s_j - s_yi + margin)))
                                         (j != yi)

    regularization penalty = sum (sum ((W_k,l)**2))
                             (k)   (l)    

    """
    


In [89]:
if __name__ == "__main__":
    main()


In epoch 0, loss is 9.003714240156306
In epoch 0 accuracy is 0.08825
In epoch 1, loss is 9.003714240156306
In epoch 1 accuracy is 0.08825
In epoch 2, loss is 9.003714240156306
In epoch 2 accuracy is 0.08825
